In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker

# 导入自己封装的库
# from Gaussian_CDF_qiyun import Gaussian_CDF,Q_Gaussian_CDF
from SARSA_function_qiyun import get_file_list

In [2]:
def get_lickNolick_reward_values(df,stage_index):
    '''
    Reward Encoding Based on Mouse Actions and Categorization of Mouse Behavior:

    mouse_actions: Mouse actions data, including information about actions such as lick and nolick.
    This section involves encoding rewards based on the mouse's actions and categorizing behaviors such as lick and nolick.
    '''
    R = 0
    lickSig = 2
    if df.loc[stage_index,'stimulus']==1:     # GO_cue
        
        if df.loc[stage_index,'outcome'] ==1:      # Hit
            R = 1.0
            lickSig = 1                                # 小鼠行为标志位
        elif df.loc[stage_index,'outcome'] == 2:   # Miss
            R = 0.1
            lickSig = 0
        elif df.loc[stage_index,'outcome'] == 3:   # CR
            R = 0.1
            lickSig = 0
        elif df.loc[stage_index,'outcome'] == 4:   # FA
            R = -2.0
            lickSig = 1
        elif df.loc[stage_index,'outcome'] == 5:   # RO
            R = -0.1
            lickSig = 1

    elif df.loc[stage_index,'stimulus']==0:     # No_GO_cue
        
        if df.loc[stage_index,'outcome'] ==1:      # Hit
            R = 1.0
        elif df.loc[stage_index,'outcome'] == 2:   # Miss
            R = 0.1
        elif df.loc[stage_index,'outcome'] == 3:   # CR
            R = 0.1
        elif df.loc[stage_index,'outcome'] == 4:   # FA
            R = -2.0
        elif df.loc[stage_index,'outcome'] == 5:   # RO
            R = -0.1
    
    return R,lickSig

In [3]:
# def get_lickNolick_Q_list(df_go,stagename):
#     '''
#     df: dataframe 传入数据行为每一次试验,column0:每次试验所给的信号,column1:为每次输出的结果或者动作.
#     N_STATES: 所切换的状态的总数,如试验的总次数.
#     ALPHA: float 取值为0~1,学习率
#     GAMMA: float 取值为0~1,折扣率
#     stage_name: 获取Q值的阶段,如stable,uncertain,reverse
#     return Q_go,Q_nogo: 返回单步更新的 Q_go,Q_nogo 值
#     '''

#     Q_go = []                         # 创建存储迭代过程的go_Q值空列表

#     # 初始化迭代初值
#     if stagename == 'stable':
#         Qs_go_lick = 1.0
#         Qs_go_nolick = 0.1

#         ALPHA = 0.1
#         GAMMA = 0
#     elif stagename == 'uncertain':
#         Qs_go_lick = 1.0
#         Qs_go_nolick = 0.1
#         ALPHA = 0.1
#         GAMMA = 0

#     elif stagename == 'reverse':
#         Qs_go_lick = 1.0
#         Qs_go_nolick = 0.1
#         ALPHA = 0.1
#         GAMMA = 0
    
#     elif stagename == 'pre_training':
#         Qs_go_lick = 0
#         Qs_go_nolick = 0
#         ALPHA = 0.4
#         GAMMA = 0

#     Q_go.append([Qs_go_lick,Qs_go_nolick])
#     go_N_STATES = np.array(df_go).shape[0]

#     for i in range(go_N_STATES-1):
        
#         R,lickSig = get_lickNolick_reward_values(df_go,i)
#         Qs_go_next_R,next_lickSig = get_lickNolick_reward_values(df_go,i+1)
        
#         if lickSig ==1:
#             Qs_go_lick = Qs_go_lick + ALPHA*(R + GAMMA * Qs_go_next_R - Qs_go_lick)
#         else:
#             Qs_go_nolick = Qs_go_nolick + ALPHA*(R + GAMMA * Qs_go_next_R - Qs_go_nolick)

#         # Qs_go = Qs_go + ALPHA*(R + GAMMA * Qs_go_next_R - Qs_go)

#         Q_go.append([Qs_go_lick,Qs_go_nolick])

#     return Q_go,Q_go[0],ALPHA,GAMMA

In [4]:
def get_lickNolick_Q_list_and_alpha(df_go,stagename):
    '''
    df: a dataframe containing data for each trial, column0: signals given for each trial, column1: results or actions taken for each trial.
    N_STATES: the total number of states to transition, such as the total number of trials.
    ALPHA: float, ranging from 0 to 1, learning rate.
    GAMMA: float, ranging from 0 to 1, discount rate.
    stage_name: the stage to retrieve Q values, such as stable, uncertain, reverse.
    return Q_go, Q_nogo: returns the Q_go, Q_nogo values for single-step updates.
    '''
    # Iterative hyperparameter settings
    THETA = 0.08
    THETA_p = 0.6
    PE = 0

    Q_go = []                         
    alpha_list = []                   
    pre_err = []                      

    # Initialize iterative initial values
    if stagename == 'stable':

        Qs_go_lick = 0.98
        Qs_go_nolick = 0.028

        ALPHA = 0.1
        # GAMMA = 0.1

    elif stagename == 'uncertain':
        Qs_go_lick = 0.98
        Qs_go_nolick = 0.028
        ALPHA = 0.1
        # GAMMA = 0.1

    elif stagename == 'reverse':
        Qs_go_lick = 0.98
        Qs_go_nolick = 0.028
        ALPHA = 0.1 
        # THETA = 0.2
        # GAMMA = 0.1

    elif stagename == 'pre_training':
        Qs_go_lick = 0
        Qs_go_nolick = 0
        ALPHA = 0.1
        GAMMA = 0.1

    Q_go.append([Qs_go_lick,Qs_go_nolick])
    alpha_list.append(ALPHA)
    pre_err.append(PE)

    go_N_STATES = np.array(df_go).shape[0]

    for i in range(go_N_STATES-1):
    # for i in range(go_N_STATES):
        
        
        R,lickSig = get_lickNolick_reward_values(df_go,i)

        if lickSig ==1:
            PE= R - Qs_go_lick
            ALPHA = THETA_p*ALPHA+THETA*abs(PE)
            Qs_go_lick = Qs_go_lick + ALPHA*(R - Qs_go_lick)
            
        else:
            PE = R - Qs_go_nolick
            ALPHA = THETA_p*ALPHA+THETA*abs(PE)
            Qs_go_nolick = Qs_go_nolick + ALPHA*(R - Qs_go_nolick)


        alpha_list.append(ALPHA)
        pre_err.append(PE)
        Q_go.append([Qs_go_lick,Qs_go_nolick])

    # Q_go.append([Qs_go_lick,Qs_go_nolick])
    # alpha_list.append(ALPHA)
    # pre_err.append(PE)

    return Q_go,Q_go[0],alpha_list,pre_err

### QP Transformation Function

In [5]:
# def softmax(Q):
#     exp_values = np.exp(Q*2.45)
#     P_sum = exp_values[:,0]+exp_values[:,1]
#     P_lick = exp_values[:,0] / P_sum
#     P_Nolick = exp_values[:,1] / P_sum
#     P = np.array([P_lick,P_Nolick])
#     # probabilities = exp_values / np.sum(exp_values,axis=0)
#     # print(probabilities.shape)
#     return P

def softmax_update(Q, st_name):
    if st_name == 'reverse':
        scale_factor=1.68
    elif st_name == 'uncertain':
        scale_factor=2.3
    elif st_name == 'stable':
        scale_factor=2.3
    elif st_name == 'pre_training':
        scale_factor=2.3
    e_Q = np.exp(scale_factor * (Q - np.max(Q)))
    P = e_Q / e_Q.sum(axis=1, keepdims=True)
    return P

# def sigmoid11(Q, scale_factor=2.4):
#     return 1 / (1 + np.exp(-scale_factor * (Q - np.max(Q))))

# def softmax_sigmoid(Q, scale_factor=2.4):
#     s = sigmoid11(Q, scale_factor)
#     P = np.stack([s, 1 - s], axis=1)
#     return P

### 概率变化绘图函数

In [6]:
# def prob_plot(prob):
#     '''
#     prob : pd.dataframe 分别有两列 分别为['lick_P']['Nolick_P']
#     根据 prob 绘制散点图并通过样条曲线添加拟合趋势线
#     '''
#     # 添加试验顺序列
#     prob['trial'] = np.arange(1, len(prob) + 1)

#     # 绘制散点图
#     plt.scatter(prob['trial'], prob['lick_P'], label='lick_P')
#     plt.scatter(prob['trial'], prob['Nolick_P'], label='Nolick_P')

#     # 使用样条曲线拟合数据
#     x_smooth = np.linspace(prob['trial'].min(), prob['trial'].max(), 200)
#     lick_spline = make_interp_spline(prob['trial'], prob['lick_P'])
#     Nolick_spline = make_interp_spline(prob['trial'], prob['Nolick_P'])
#     lick_smooth = lick_spline(x_smooth)
#     Nolick_smooth = Nolick_spline(x_smooth)

#     # 绘制样条曲线
#     plt.plot(x_smooth, lick_smooth, 'r-', label='lick_P spline')
#     plt.plot(x_smooth, Nolick_smooth, 'g-', label='Nolick_P spline')

#     # 在图中添加样条曲线的表达式
#     # lick_expr = r'$\text{{lick\_P spline}}: f(x)$'
#     # Nolick_expr = r'$\text{{Nolick\_P spline}}: g(x)$'
#     # plt.text(2, 0.4, lick_expr, fontsize=12, color='red')
#     # plt.text(2, 0.8, Nolick_expr, fontsize=12, color='green')

#     plt.xlabel('Trial')
#     plt.ylabel('Probability')
#     plt.legend()
#     plt.show()

### pretraining阶段的Q值数据

In [7]:
# stage_name = ['pre_training']

# stage_total_tab = []
# for j in range(len(stage_name)):
#     file_list = get_file_list(r'E:\project_wuqiyun\Pyddm_wuqiyun\原始数据\ACC_TRAINING_cwq\pre_training_qiyun\pre_traning_mice_selcect_data')
#     mice_total_tab = []
#     for i in range(len(file_list)):
#         df = pd.read_csv(file_list[i])
#         df = df.loc[:, ['stimulus', 'outcome']]
#         df_go = df[df['stimulus'] == 1]   # go 的 Q 值

#         df_go.reset_index(inplace=True)

#         # stable_Q_go, init_Qs_go, ALPHA, GAMMA = get_lickNolick_Q_list(df_go, stage_name[j])
#         stable_Q_go, init_Qs_go, alpha_list,pre_err = get_lickNolick_Q_list_and_alpha(df_go, stage_name[j])
        
#         stable_Q_go = pd.DataFrame(stable_Q_go, columns=['Lick_Q', 'NoLick_Q'])  # Convert to DataFrame
        
#         Q_go_array = np.array(stable_Q_go)
#         # P = softmax(Q_go_array).T
#         P = softmax_update(Q_go_array,'pre_training')

#         prob = pd.DataFrame(P, columns=['lick_P', 'Nolick_P'])
#         dff = pd.concat([df_go,stable_Q_go,prob],axis=1)
#         # dff = pd.concat([df_go,stable_Q_go],axis=1)

#         dff.drop(columns='index',axis=1,inplace=True)
#         # 增加动态的学习率和PE进去
#         dff['Alpha'] = alpha_list
#         dff['PE'] = pre_err
        
#         mice_total_tab.append(dff)

#     stage_total_tab.append(mice_total_tab)
    
# for hr in range(len(stage_total_tab[0])):
#     dx = stage_total_tab[0][hr]
#     dx.to_csv(f'E:\project_wuqiyun\Pyddm_wuqiyun\RL_SARSA_Q_data_update_qiyun\experimental_data\pretaining\qiyun_pre_training_mice{hr+1}.csv')
# stage_total_tab[0][0]

### stable uncertain reverse 三个阶段的Q值数据

In [8]:
stage_name = ['stable', 'uncertain', 'reverse']
# stage_name = ['pre_training']

fig11, axes = plt.subplots(len(stage_name), 6, figsize=(30, 10), dpi=500)
stage_total_tab = []
for j in range(len(stage_name)):
    file_list = get_file_list(r'G:\new_data_analysis\behavior_new\{}'.format(stage_name[j]))
    mice_total_tab = []
    for i in range(len(file_list)):
        df = pd.read_csv(file_list[i])
        df = df.loc[:, ['stimulus', 'outcome']]
        df_go = df[df['stimulus'] == 1]   # The Q values associated with the "go" signal.

        df_go.reset_index(inplace=True)

        # stable_Q_go, init_Qs_go, ALPHA, GAMMA = get_lickNolick_Q_list(df_go, stage_name[j])
        # stable_Q_go = pd.DataFrame(stable_Q_go, columns=['Lick_Q', 'NoLick_Q'])  # Convert to DataFrame

        stable_Q_go, init_Qs_go, alpha_list,pre_err = get_lickNolick_Q_list_and_alpha(df_go, stage_name[j])
        stable_Q_go = pd.DataFrame(stable_Q_go, columns=['Lick_Q', 'NoLick_Q'])  # Convert to DataFrame
        
        Q_go_array = np.array(stable_Q_go)
        # P = softmax(Q_go_array).T
        P = softmax_update(Q_go_array,stage_name[j])

        prob = pd.DataFrame(P, columns=['lick_P', 'Nolick_P'])
        dff = pd.concat([df_go,stable_Q_go,prob],axis=1)
        # dff = pd.concat([df_go,stable_Q_go],axis=1)

        dff.drop(columns='index',axis=1,inplace=True)
        # 增加动态的学习率和PE进去
        dff['Alpha'] = alpha_list
        dff['PE'] = pre_err
        
        mice_total_tab.append(dff)

        # =====================Plotting Section=================================

        ax = axes[j, i]

        # 使用样条曲线拟合
        x_smooth = np.linspace(0, len(prob), 200)
        spline_lick = make_interp_spline(range(len(prob)), prob['lick_P'])
        spline_nolick = make_interp_spline(range(len(prob)), prob['Nolick_P'])
        y_lick_smooth = spline_lick(x_smooth)
        y_nolick_smooth = spline_nolick(x_smooth)

        ax.plot(x_smooth, y_lick_smooth, label='lick_P')
        ax.plot(x_smooth, y_nolick_smooth, label='Nolick_P')

        ax2 = ax.twinx()  # Create a secondary y-axis

        ax2.scatter(range(len(stable_Q_go)), stable_Q_go['Lick_Q'], color='r', label='Lick_Q')
        ax2.scatter(range(len(stable_Q_go)), stable_Q_go['NoLick_Q'], color='g', label='NoLick_Q')

        ax.set_xlabel('Trial')
        ax.set_ylabel('Probability')
        ax.grid()
        ax2.set_ylabel('Q-values')

        ax.set_title('Mouse {}, Stage: {}'.format(i+1, stage_name[j]))
        ax.legend(loc='upper left')
        ax2.legend(loc='upper right')

    stage_total_tab.append(mice_total_tab)
    
plt.tight_layout()
plt.show()
# 保存图片到本地
# fig11.savefig(r'E:\project_wuqiyun\Pyddm_wuqiyun\RL_SARSA_Q_data_update_qiyun\figure\各个阶段每只小鼠的真实Q值变化图.png', dpi=300)

In [9]:
# # 三个阶段的学习率比较
# stage_alpha_mean = {}
# fig22, axis = plt.subplots(1,3,figsize =(15,3))
# for sta in range(3):
#     sumsum = []
#     for su in range(len(stage_total_tab[sta])):
#         dt = stage_total_tab[sta][su]
#         if sta != 2:
#             dg = dt['Alpha'].mean()
#         else:
#             dg = dt['Alpha'][:10].mean()

#         rr = (dt['Alpha'])*(dt['PE'])
#         # print(rr)
#         # plt.plot(dt.index,rr)
#         # axis[sta].plot(dt.index,dt['Alpha'],label=f'mice{su+1}')
#         # axis[sta].plot(dt.index,dt['PE'],label=f'mice{su+1}')

#         axis[sta].plot(dt.index,rr,label=f'mice{su+1}')

#         axis[sta].set_xlabel('Trial')
#         # axis[sta].set_ylabel('Alpha')
#         # axis[sta].set_ylabel('PE')
#         axis[sta].set_ylabel('Alpha*PE')

#         axis[sta].set_title('Stage: {}'.format(stage_name[sta]))
#         axis[sta].legend()

#         sumsum.append(dg)
#     alpha_mean = np.mean(sumsum)
#     stage_alpha_mean[stage_name[sta]] = alpha_mean
# print('三个阶段,stable,uncertain,及reverse前10次的学习率变化均值')
# stage_alpha_mean

# # 将图片保存至本地
# fig22.savefig(r'E:\project_wuqiyun\Pyddm_wuqiyun\RL_SARSA_Q_data_update_qiyun\figure\stable,uncertain,及reverse前10次的alpha及PE变化.png',dpi=300)

### 将数据保存到本地

In [17]:
# stage_na = ['stable', 'uncertain', 'reverse']
stage_na = ['stable', 'uncertain', 'reverse']

for i in range(len(stage_total_tab)):
    for j in range(len(stage_total_tab[i])):
        dfg = stage_total_tab[i][j] # Output data for each stage and each mouse to a local folder
        dfg.to_csv(f'G:\\new_data_analysis\\behavior_new\experimental_data\\qiyun_{stage_na[i]}_mice{j+1}.csv')

In [11]:
# stage_name = ['stable', 'uncertain', 'reverse']
# fig33, axs = plt.subplots(len(stage_total_tab), 6, figsize=(40, 15), dpi=500)
# label_dict = {1: 'Hit', 2: 'Miss', 3: 'CR', 4: 'FA', 5: 'RO', 6: 'UR'}

# for i in range(len(stage_total_tab)):
#     for j in range(len(stage_total_tab[i])):
#         # 创建子图
#         ax = axs[i][j]
        
#         for label, color in zip(range(1, 6), ('blue', 'yellow', 'orange', 'red', 'green')):
#             # 绘制散点图
#             ax.scatter(stage_total_tab[i][j].index[stage_total_tab[i][j]['outcome'] == label], 
#                        stage_total_tab[i][j]['Lick_Q'][stage_total_tab[i][j]['outcome'] == label],
#                        label=label_dict[label], color=color)
        
#         # 设置子图属性
#         ax.legend(loc='upper right')
#         ax.set_xlabel('Trial')
#         ax.set_ylabel('Lick_Q')
#         ax.set_title('Mouse {}, Stage: {}'.format(j+1, stage_name[i]))
#         ax.grid()
        
#         # 设置y轴刻度为0.1为单位
#         ax.yaxis.set_major_locator(ticker.MultipleLocator(0.2))
        
#         # 设置y轴范围为-1.2到+1.2
#         ax.set_ylim(-1.5, 1.2)

# # 显示图形
# plt.show()

# # 将图片保存至本地
# fig33.savefig(r'E:\project_wuqiyun\Pyddm_wuqiyun\RL_SARSA_Q_data_update_qiyun\figure\各个阶段每只小鼠的lick_Q及刺激类型对应图.png', dpi=300)

In [12]:
# import matplotlib.pyplot as plt
# from scipy.interpolate import make_interp_spline

# stage_name = ['stable', 'uncertain', 'reverse']
# fig, axes = plt.subplots(len(stage_name), 6, figsize=(30, 10),dpi=500)
# for j in range(len(stage_name)):
#     file_list = get_file_list(r'E:\project_wuqiyun\Pyddm_wuqiyun\原始数据\{}'.format(stage_name[j]))
#     for i in range(len(file_list)):
#         df = pd.read_csv(file_list[i])
#         df = df.loc[:, ['stimulus', 'outcome']]
#         df_go = df[df['stimulus'] == 1]
#         df_nogo = df[df['stimulus'] == 0]
#         df_go.reset_index(inplace=True)
#         df_go.drop(columns='index', axis=1, inplace=True)
#         df_nogo.reset_index(inplace=True)
#         df_nogo.drop(columns='index', axis=1, inplace=True)

#         stable_Q_go, init_Qs_go, ALPHA, GAMMA = get_lickNolick_Q_list(df_go, stage_name[j])

#         Q_go_array = np.array(stable_Q_go)
#         P = softmax(Q_go_array).T
#         prob = pd.DataFrame(P, columns=['lick_P', 'Nolick_P'])

#         ax = axes[j, i]
#         ax.scatter(range(len(prob)), prob['lick_P'], label='lick_P')
#         ax.scatter(range(len(prob)), prob['Nolick_P'], label='Nolick_P')

#         # 使用样条曲线拟合
#         x_smooth = np.linspace(0, len(prob), 200)
#         spline_lick = make_interp_spline(range(len(prob)), prob['lick_P'])
#         spline_nolick = make_interp_spline(range(len(prob)), prob['Nolick_P'])
#         y_lick_smooth = spline_lick(x_smooth)
#         y_nolick_smooth = spline_nolick(x_smooth)

#         ax.plot(x_smooth, y_lick_smooth, label='lick_P Trend')
#         ax.plot(x_smooth, y_nolick_smooth, label='Nolick_P Trend')
#         ax.grid()
#         ax.set_xlabel('Trial')
#         ax.set_ylabel('Probability')
#         ax.set_title('Mouse {},Stage: {}'.format(i+1,stage_name[j]))
#         ax.legend()

# plt.tight_layout()
# plt.show()
# plt.title('DifferentMice_ThreeStage_lickAndNoLick_Prob')



### 模拟决策序列生成函数调试

In [13]:

# # 没有增加线性比例缩放项的QP映射函数
# # def qiyun_softmax(x):
# #     """Softmax function to convert Q-values to probabilities"""
# #     e_x = np.exp(x - np.max(x))
# #     return e_x / e_x.sum()

# # def qiyun_softmax(x, scale_factor=2.45):
# #     """Softmax function with linear scaling factor"""
# #     e_x = np.exp(scale_factor * (x - np.max(x)))
# #     return e_x / e_x.sum()


# def sigmoid_prob(data, scale_factor=2.3):
#     """Convert lick and nolick values to probabilities using sigmoid function"""
#     lick_prob = 1 / (1 + np.exp(-scale_factor * (data[0] - data[1])))
#     nolick_prob = 1 - lick_prob
#     return lick_prob, nolick_prob


# def generate_action(init_lick_Q,init_Nolick_Q,action_len,Alpha,Gamma):

#     '''
#     init_lick_Q :lick 初始 Q 值
#     init_Nolick_Q :nolick 初始 Q 值
#     action_len: 模拟生成的行动序列长度
#     '''

#     # Set initial Q-values
#     lick_q = init_lick_Q
#     nolick_q = init_Nolick_Q

#     # Iteration count
#     N = action_len

#     # Learning rate and discount factor
#     alpha = Alpha
#     gamma = Gamma

#     # Create an empty DataFrame to record data
#     columns = ['Lick Q', 'NoLick Q', 'Lick Probability', 'NoLick Probability', 'Action']
#     data = pd.DataFrame(columns=columns)

#     # Iterate to generate data
#     for i in range(N):
#         # Convert Q-values to probabilities
#         lick_prob = sigmoid_prob([lick_q, nolick_q])[0]
#         nolick_prob = sigmoid_prob([lick_q, nolick_q])[1]
#         # print(lick_prob)
#         # Choose action based on probabilities
#         action = np.random.choice(['Lick', 'NoLick'], p=[lick_prob, nolick_prob])
#         action_next = np.random.choice(['Lick', 'NoLick'], p=[lick_prob, nolick_prob])
#         if action_next == 'Lick':
#             # 10% probability of setting reward to 0
#             if np.random.random() <= 0.2:
#                 action_next = 'RO'
#                 reward_next = 0
#             else:
#                 reward_next = 1
#         elif action_next == 'NoLick':
#             reward_next = 0

#         # print(action)
#         # Update Q-values
#         if action == 'Lick':
#             # 10% probability of setting reward to 0
#             if np.random.random() <= 0.2:
#                 action = 'RO'
#                 reward = 0
#             else:
#                 reward = 1

#             # Record data
#             data.loc[i] = [lick_q, nolick_q, lick_prob, nolick_prob, action]
            
#             # Update Q-values using learning rate and discount factor
#             new_lick_q = lick_q + alpha * (reward + gamma * reward_next - lick_q)
            
#             new_nolick_q = nolick_q
#             # new_lick_q = lick_q + alpha * (reward + gamma * (lick_prob)) - lick_q)
#             # new_nolick_q = nolick_q + alpha * (0 + gamma * nolick_q - nolick_q)

#             lick_q, nolick_q = new_lick_q, new_nolick_q
            
#         elif action == 'NoLick':
#             reward = 0
            
#             # Record data
#             data.loc[i] = [lick_q, nolick_q, lick_prob, nolick_prob, action]
            
#             # Update Q-values using learning rate and discount factor
#             new_lick_q = lick_q 
#             new_nolick_q = nolick_q + alpha * (0 + gamma * reward_next - nolick_q)
            
#             lick_q, nolick_q = new_lick_q, new_nolick_q
#     return data